In [4]:
import numpy as np
import skfuzzy as fuzz
from skfuzzy import control as ctrl
import pandas as pd
from sklearn.metrics import accuracy_score, f1_score

# 1. Baca data dari CSV
df = pd.read_csv('../data/students_data.csv')

# 2. Prapemrosesan: cek missing value, isi dengan rata-rata jika ada
df = df.fillna(df.mean(numeric_only=True))

# 3. Definisikan sistem fuzzy Mamdani
quiz = ctrl.Antecedent(np.arange(0, 401, 1), 'quiz')
exam = ctrl.Antecedent(np.arange(0, 401, 1), 'exam')
assignment = ctrl.Antecedent(np.arange(0, 401, 1), 'assignment')
final_score = ctrl.Consequent(np.arange(0, 101, 1), 'final_score')

quiz['low'] = fuzz.trimf(quiz.universe, [0, 0, 200])
quiz['medium'] = fuzz.trimf(quiz.universe, [100, 200, 300])
quiz['high'] = fuzz.trimf(quiz.universe, [200, 400, 400])
exam['low'] = fuzz.trimf(exam.universe, [0, 0, 200])
exam['medium'] = fuzz.trimf(exam.universe, [100, 200, 300])
exam['high'] = fuzz.trimf(exam.universe, [200, 400, 400])
assignment['low'] = fuzz.trimf(assignment.universe, [0, 0, 200])
assignment['medium'] = fuzz.trimf(assignment.universe, [100, 200, 300])
assignment['high'] = fuzz.trimf(assignment.universe, [200, 400, 400])

final_score['F'] = fuzz.trimf(final_score.universe, [0, 0, 50])
final_score['D'] = fuzz.trimf(final_score.universe, [40, 50, 60])
final_score['C'] = fuzz.trimf(final_score.universe, [50, 60, 70])
final_score['B'] = fuzz.trimf(final_score.universe, [65, 75, 85])
final_score['A'] = fuzz.trimf(final_score.universe, [80, 100, 100])

rule1 = ctrl.Rule(quiz['high'] & exam['high'] & assignment['high'], final_score['A'])
rule2 = ctrl.Rule(quiz['medium'] & exam['high'] & assignment['medium'], final_score['B'])
rule3 = ctrl.Rule(quiz['medium'] & exam['medium'] & assignment['medium'], final_score['C'])
rule4 = ctrl.Rule(quiz['low'] | exam['low'] | assignment['low'], final_score['F'])
rule5 = ctrl.Rule(quiz['medium'] & exam['low'] & assignment['medium'], final_score['D'])

final_ctrl = ctrl.ControlSystem([rule1, rule2, rule3, rule4, rule5])
final_sim = ctrl.ControlSystemSimulation(final_ctrl)

# 4. Definisikan sistem fuzzy Sugeno
def sugeno_fuzzy(quiz_val, exam_val, assignment_val):
    def low(x):
        return max(min((200 - x)/200, 1), 0) if x <= 200 else 0
    def medium(x):
        if 100 < x < 200:
            return (x - 100)/100
        elif 200 <= x <= 300:
            return (300 - x)/100
        else:
            return 0
    def high(x):
        return max(min((x - 200)/200, 1), 0) if x >= 200 else 0

    quiz_level = {'low': low(quiz_val), 'medium': medium(quiz_val), 'high': high(quiz_val)}
    exam_level = {'low': low(exam_val), 'medium': medium(exam_val), 'high': high(exam_val)}
    assignment_level = {'low': low(assignment_val), 'medium': medium(assignment_val), 'high': high(assignment_val)}

    def weighted_score(q, e, a):
        return (q/400)*20 + (e/400)*50 + (a/400)*30

    rules = [
        (min(quiz_level['high'], exam_level['high'], assignment_level['high']), weighted_score),
        (min(quiz_level['medium'], exam_level['high'], assignment_level['medium']), weighted_score),
        (min(quiz_level['medium'], exam_level['medium'], assignment_level['medium']), weighted_score),
        (max(quiz_level['low'], exam_level['low'], assignment_level['low']), weighted_score),
        (min(quiz_level['medium'], exam_level['low'], assignment_level['medium']), weighted_score),
    ]

    numerator = 0
    denominator = 0
    for weight, func in rules:
        if weight > 0:
            f_val = func(quiz_val, exam_val, assignment_val)
            numerator += weight * f_val
            denominator += weight
    if denominator == 0:
        return 0
    return numerator / denominator

# 5. Fungsi konversi nilai ke grade
def nilai_ke_grade(nilai):
    if nilai >= 80:
        return "A"
    elif nilai >= 70:
        return "AB"
    elif nilai >= 65:
        return "B"
    elif nilai >= 60:
        return "BC"
    elif nilai >= 50:
        return "C"
    elif nilai >= 40:
        return "D"
    else:
        return "E"

# 6. Membuat label acuan (manual) dari rata-rata nilai (skala 0-100)
def manual_grade(q, e, a):
    total = (q + e + a) / 12  # Skala ke 0-100
    if total >= 80:
        return "A"
    elif total >= 70:
        return "AB"
    elif total >= 65:
        return "B"
    elif total >= 60:
        return "BC"
    elif total >= 50:
        return "C"
    elif total >= 40:
        return "D"
    else:
        return "E"

df['grade_manual'] = df.apply(lambda row: manual_grade(row['Quiz'], row['Exam'], row['Assignment']), axis=1)

# 7. Prediksi Mamdani & Sugeno untuk seluruh data
pred_mamdani = []
pred_sugeno = []

for _, row in df.iterrows():
    # Mamdani
    final_sim.input['quiz'] = row['Quiz']
    final_sim.input['exam'] = row['Exam']
    final_sim.input['assignment'] = row['Assignment']
    final_sim.compute()
    nilai_mamdani = final_sim.output['final_score']
    pred_mamdani.append(nilai_ke_grade(nilai_mamdani))
    # Sugeno
    nilai_sugeno = sugeno_fuzzy(row['Quiz'], row['Exam'], row['Assignment'])
    pred_sugeno.append(nilai_ke_grade(nilai_sugeno))

df['grade_mamdani'] = pred_mamdani
df['grade_sugeno'] = pred_sugeno

# 8. Evaluasi
print("Akurasi Mamdani:", accuracy_score(df['grade_manual'], df['grade_mamdani']))
print("F1-score Mamdani:", f1_score(df['grade_manual'], df['grade_mamdani'], average='macro'))
print("Akurasi Sugeno:", accuracy_score(df['grade_manual'], df['grade_sugeno']))
print("F1-score Sugeno:", f1_score(df['grade_manual'], df['grade_sugeno'], average='macro'))

# 9. Tampilkan hasil prediksi dan label acuan
print(df[['Quiz', 'Exam', 'Assignment', 'grade_manual', 'grade_mamdani', 'grade_sugeno']])

Akurasi Mamdani: 0.375
F1-score Mamdani: 0.16066000868432478
Akurasi Sugeno: 0.55
F1-score Sugeno: 0.5194045085349434
    Quiz  Exam  Assignment grade_manual grade_mamdani grade_sugeno
0    379   348         167           AB             E           AB
1    145    34          22            E             E            E
2     27   397         101            D             E            C
3    330   382         282            A             A            A
4    279    12          14            E             E            E
5    108   300         251            C             E           BC
6    111   318         323           BC             E            B
7     82   241          15            E             E            E
8     35   127         288            E             E            E
9    191   226          16            E             E            E
10    64     3         301            E             E            E
11   267    12         207            D             E            E
12   279   